In [ ]:
import dspy

lm = dspy.OllamaLocal(model="llama3", base_url="http://localhost:11434")

dspy.settings.configure(lm=lm)

In [ ]:
create_post_analysis = dspy.Predict("journal_entry -> therapeutic_advice")

journal_entry = "I think I have been angry about the ways I've been treated in my family system. I need some tools to help myself calm downin the heat of the moment."

response = create_post_analysis(journal_entry=journal_entry)

print(response.therapeutic_advice)

In [ ]:
import weaviate
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import contextlib

import os

WEAVIATE_COLLECTION_NAME = (
        os.getenv("WEAVIATE_COLLECTION_NAME") or "WeaviateJournalChunks"
    )
WEAVIATE_HOST = os.getenv("WEAVIATE_HOST") or "localhost"

def get_collection():

    client = weaviate.connect_to_local(host=WEAVIATE_HOST)
    collection = client.collections.get(WEAVIATE_COLLECTION_NAME)

    try:
        yield collection
    finally:
        client.close()

@contextlib.contextmanager
def get_weaviate_client():
    client = weaviate.connect_to_local(host=WEAVIATE_HOST)

    try:
        yield client
    finally:
        client.close()

with get_weaviate_client() as weaviate_client:


    retriever_model = WeaviateRM(
        weaviate_collection_name=WEAVIATE_COLLECTION_NAME,
        weaviate_client=weaviate_client 
    )

    results = retriever_model("Explore the significance of quantum computing", k=2)

    for result in results:
        print("Document:", result.long_text, "\n")

In [ ]:
weaviate_client.close()